In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
PP链核反应网络示例代码（包含 pp, ³He–³He, ³He–⁴He 分支，以及后续的 ⁷Be 衰变与后续反应）
参考：Caughlan & Fowler 1988 中的速率参数化（数值仅供演示）
"""

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Avogadro 数
NA = 6.022e23

# 反应速率参数化函数（单位：cm^3/mol/s，对于两体反应，实际速率常数需要除以 NA）
def rate_pp(T9):
    # p + p -> d + e+ + ν
    return 4.01e-15 * T9**(-2/3) * np.exp(-3.380 * T9**(-1/3)) 

def rate_dp(T9):
    # d + p -> ³He + γ
    return 2.24e3 * T9**(-2/3) * np.exp(-3.720 * T9**(-1/3)) 

def rate_33(T9):
    # ³He + ³He -> ⁴He + 2p
    return 6.04e10 * T9**(-2/3) * np.exp(-12.276 * T9**(-1/3)) 

def rate_34(T9):
    # ³He + ⁴He -> ⁷Be + γ
    return 5.61e6 * (T9 / (1 + 4.95e-2 * T9))**(5/6) / (T9 ** (2/3)) * np.exp(-12.826 / ((T9 / (1 + 4.95e-2 * T9)) ** (1/3)))

def rate_li7p(T9):
    # ⁷Li + p -> 2⁴He
    return 1.096e9 * T9**(-2/3) * np.exp(-8.472 * T9**(-1/3))

def rate_be7p(T9):
    # ⁷Be + p -> ⁸B + γ
    return 3.11e5 * T9**(-2/3) * np.exp(-10.262 * T9**(-1/3))

# 其它速率（直接给出 s^-1 数值）
#lambda_be7e = 1e-7          # ⁷Be + e- -> ⁷Li + ν  （电子捕获）
#lambda_b8_decay = np.log(2)/0.77   # ⁸B 衰变，半衰期约 0.77 s

N_A = 6.022e23  

def T9A(T9):
    """计算修正温度 T9A"""
    return T9 / (1 + 4.95e-2 * T9)

def reaction_rate(T9):
    """计算反应速率 (sigma * v)"""
    T9A_value = T9A(T9)
    rate = 5.61e6 * (T9A_value**5.6) / (T9**3.2) * np.exp(-12.826 / (T9A_value**1.3))
    return rate

def lambda_rate(T9):
    """计算速率常数 λ = N_A * reaction_rate"""
    return N_A * reaction_rate(T9)

# 示例计算
T9_sample = 1.5  # 假设 T9 = 1.5
lambda_value = lambda_rate(T9_sample)
print(f"At T9 = {T9_sample}, lambda rate = {lambda_value:.4e} s^-1")

In [ ]:
# 假定恒定温度与密度
T = 1.5e7         # K
T9 = T / 1e9      # 单位为 10^9 K

nH0 = 6e26
# 初始核心中氢的数密度 [cm^-3]

# 根据文献给出的公式，将两体反应速率转换为 cm^3/s（除以 NA）
lambda_pp   = rate_pp(T9)   / NA    # p+p
lambda_dp   = rate_dp(T9)   / NA    # d+p
lambda_33   = rate_33(T9)   / NA    # ³He+³He
lambda_34   = rate_34(T9)   / NA    # ³He+⁴He
lambda_li7p = rate_li7p(T9) / NA    # ⁷Li+p
lambda_be7p = rate_be7p(T9) / NA    # ⁷Be+p

# 反应网络中包含的物种：
# y[0] : p (氢)
# y[1] : d (氘)
# y[2] : ³He
# y[3] : ⁴He
# y[4] : ⁷Be
# y[5] : ⁷Li
# y[6] : ⁸B
#
# 初始条件：初始时主要为 p 和 ⁴He（取 ⁴He/n_p ~ 0.1），其它中间产物初值取 0
y0 = [ nH0,       # p
       0.0,       # d
       0.0,       # ³He
       0.1*nH0,   # ⁴He
       0.0,       # ⁷Be
       0.0,       # ⁷Li
       0.0 ]      # ⁸B

# 定义反应网络的微分方程（单位：cm^-3/s）
def dY_dt(t, y):
    # 各物种数密度
    p   = y[0]
    d   = y[1]
    He3 = y[2]
    He4 = y[3]
    Be7 = y[4]
    Li7 = y[5]
    B8  = y[6]
    
    # 反应速率（cm^-3 s^-1）
    # p+p 反应：注意由于两个相同粒子需乘以 0.5 对称因子
    r1 = 0.5 * lambda_pp * p * p         # p + p -> d + e+ + ν
    r2 = lambda_dp * d * p                # d + p -> ³He + γ
    r3 = 0.5 * lambda_33 * He3 * He3       # ³He + ³He -> ⁴He + 2p
    r4 = lambda_34 * He3 * He4             # ³He + ⁴He -> ⁷Be + γ
    r5 = lambda_be7e * Be7                 # ⁷Be + e- -> ⁷Li + ν
    r6 = lambda_li7p * Li7 * p             # ⁷Li + p -> 2⁴He
    r7 = lambda_be7p * Be7 * p             # ⁷Be + p -> ⁸B + γ
    r8 = lambda_b8_decay * B8              # ⁸B 衰变 -> 2⁴He
    
    # 反应网络微分方程（注意各反应消耗与生成的核数）
    dp_dt   = -2*r1 - r2 - r6 - r7 + 2*r3
    dd_dt   = + r1 - r2
    dHe3_dt = + r2 - 2*r3 - r4
    dHe4_dt = + r3 + 2*r6 + 2*r8 - r4
    dBe7_dt = + r4 - r5 - r7
    dLi7_dt = + r5 - r6
    dB8_dt  = + r7 - r8

    return [dp_dt, dd_dt, dHe3_dt, dHe4_dt, dBe7_dt, dLi7_dt, dB8_dt]

# 主程序：积分、处理数据、绘图
if __name__ == '__main__':
    # 积分时间区间（单位：s），例如积分到 1e17 s（约 3 亿年），因 PP 链反应非常缓慢
    t_start = 0.0
    t_end   = 1e17
    # 为了更好地展示变化，可取对数时间步点
    n_points = 1000
    t_eval = np.logspace(np.log10(1e0), np.log10(t_end), n_points)
    
    sol = solve_ivp(dY_dt, [t_start, t_end], y0, method='BDF', t_eval=t_eval, rtol=1e-8, atol=1e-20)
    
    if not sol.success:
        raise RuntimeError("ODE 求解失败！")
    
    # 提取结果
    t = sol.t   # 时间（s）
    y = sol.y   # 各物种数密度，行对应物种（顺序同 y0）
    
    # 计算各物种相对于 p 的比值（注意：随时间 p 也在变化，这里取 n_i/n_p）
    ratio_d   = y[1] / y[0]
    ratio_He3 = y[2] / y[0]
    ratio_He4 = y[3] / y[0]
    ratio_Be7 = y[4] / y[0]
    ratio_Li7 = y[5] / y[0]
    ratio_B8  = y[6] / y[0]
    
    # 绘图
    plt.figure(figsize=(8, 6))
    plt.loglog(t, y[0], label='p')
    plt.loglog(t, y[1], label='d')
    plt.loglog(t, y[2], label='³He')
    plt.loglog(t, y[3], label='⁴He')
    plt.loglog(t, y[4], label='⁷Be')
    plt.loglog(t, y[5], label='⁷Li')
    plt.loglog(t, y[6], label='⁸B')
    plt.xlabel("t[s]")
    plt.ylabel("density(cm$^{-3}$)")
    plt.title("Abundance Evolution")
    plt.legend()
    plt.grid(True, which="both", ls="--")
    plt.tight_layout()
    plt.show()
    
    # -------------------------------
    # 2. 打印出最终时刻的各物种 abundance
    species = ['p', 'd', '³He', '⁴He', '⁷Be', '⁷Li', '⁸B']
    print("final time t = {:.3e} s 的 abundance:".format(t[-1]))
    for i, sp in enumerate(species):
        print("{:>4}: {:.3e} cm^-3".format(sp, y[i, -1]))
    
    # -------------------------------
    # 3. 计算各物种与 p 的 ratio 并绘图
    ratio_d   = y[1] / y[0]
    ratio_He3 = y[2] / y[0]
    ratio_He4 = y[3] / y[0]
    ratio_Be7 = y[4] / y[0]
    ratio_Li7 = y[5] / y[0]
    ratio_B8  = y[6] / y[0]
    
    plt.figure(figsize=(8, 6))
    plt.loglog(t, ratio_d, label='d/p')
    plt.loglog(t, ratio_He3, label='³He/p')
    plt.loglog(t, ratio_He4, label='⁴He/p')
    plt.loglog(t, ratio_Be7, label='⁷Be/p')
    plt.loglog(t, ratio_Li7, label='⁷Li/p')
    plt.loglog(t, ratio_B8, label='⁸B/p')
    
    plt.xlabel("t[s]")
    plt.ylabel("ratio")
    plt.title("Abundance Ratio Evolution")
    plt.legend()
    plt.grid(True, which="both", ls="--")
    plt.tight_layout()
    plt.show()